[![imagenes](imagenes/pythonista.png)](https://pythonista.mx)

# Ejemplo de uso con JSON y Jinja 2.

En este capítulo se creará una aplicación que realice una búsqueda de datos:

* Regresando un objeto serializado en formato JSON.
* Desplegando una página HTML mediante el uso de plantillas de Jinja 2.

## La función *encuentra()*.

* Hace una búsqueda de un objeto tipo _str_ que se ingresa como argumento en el parámetro *cadena*, dentro de los campos pertenecientes al objeto tipo _dict_ que se ingresa como argumento para el parámetro *registro* de la función. La función sólo buscará en los campos correspondientes a los indicadores listados dentro del objeto _tuple_ o _list_ que se ingresa como argumento para el parámetro *campos*.
* En casode que la cadena ingreesada forme parte del texto contenido en al menos uno de los campos indicados, la función regresará _True_. De lo contrario, regresará _False_.


In [ ]:
encuentra = lambda cadena, registro, campos: bool(sum([cadena.casefold() \
            in registro[campo].casefold() for campo in campos]))

** Ejemplos:**

In [ ]:
encuentra('nez', {'Nombre': 'Juan', 'Apellido': 'Godínez', 'correo': 'falsonez@falso.com'},
          ['Apellido', 'Nombre'])

In [ ]:
encuentra('Juan', {'Nombre': 'Juan', 'Apellido': 'Godínez', 'correo': 'falsonez@falso.com'},
          ['correo'])

In [ ]:
encuentra('n', {'Nombre': 'Juan', 'Apellido': 'Godínez', 'correo': 'falsonez@falso.com'},
          ['correo', 'Nombre', 'Apellido'])

## La función *buscar_archivo()*.

* Abre un archivo de texto localizado en la ruta definida como argumento para el parámetro *ruta*.
* Lee y evalúa al contenido del archivo como si fuera una expresión.
* Mediante el uso de la función *encuentra()*, regresará una lista de objetos que contenga al objeto _str_ ingresado como argumento para el parámetro *cadena* en al menos uno de los campos incluidos en el parámetro *campos*.

In [ ]:
def buscar_archivo(cadena, ruta, campos):
    with open(ruta, 'tr') as archivo:
        base = eval(archivo.read())
    return [registro for registro in base if encuentra(cadena, registro, campos)]

**Ejemplo:**

El archivo [data/alumnos.txt](data/alumnos.txt) contiene la representación de un objeto tipo *list* que a suvez contiene una sucesiṕn de objetos tipo _dict_ con los identificadores _"Cuenta"_, _"Nombre"_, _"Primer Apellido"_, _"Segundo Apellido"_, _"Carrera"_, _"Semestre"_, _"Promedio"_ y _"Al Corriente"_.

```
[{'Cuenta': 1231221, 'Al Corriente': False, 'Carrera': 'Arquitectura', 'Nombre': 'Pedro', 'Primer Apellido': 'Solis', 'Promedio': 7.8, 'Semestre': 3, 'Segundo Apellido': 'Cabañas'}, 
{'Cuenta': 1231222, 'Al Corriente': False, 'Carrera': 'Actuaría', 'Nombre': 'Yolanda', 'Primer Apellido': 'Jiménez', 'Segundo Apellido': 'Lerdo', 'Promedio': 6, 'Semestre': 3}, 
{'Cuenta': 1231223, 'Al Corriente': True, 'Carrera': 'Sistemas', 'Nombre': 'Juan', 'Primer Apellido': 'Ramos', 'Segundo Apellido': 'Breña', 'Promedio': 8.6, 'Semestre': 9},
{'Cuenta': 1231224, 'Al Corriente': True, 'Carrera': 'Derecho', 'Nombre': 'Mayra Jimena', 'Primer Apellido': 'Cervantes', 'Segundo Apellido': 'Lisama', 'Promedio': 9.2, 'Semestre': 12}]
```
Se utilizará la función *buscar_archivo()* para encontrar los elementos que coincidan con la cadena de búsqueda en los campos _'Nombre'_, _'Primer Apellido'_ y _'Segundo Apellido'_ dentro del archivo *data/alumnos.txt*.

In [ ]:
campos = ('Nombre', 'Primer Apellido', 'Segundo Apellido')
ruta = 'data/alumnos.txt'

In [ ]:
buscar_archivo('ed', ruta, campos)

In [ ]:
buscar_archivo('z', ruta, campos)

In [ ]:
buscar_archivo('Derecho', ruta, campos)

In [ ]:
buscar_archivo('Ramos', ruta, campos)

## Ejemplo de búsqueda que regresa datos en JSON.

El siguiente código levantará un servicio web que regresará una búsqueda con la función *buscar_archivo()* si la cadena de búsqueda se ingresa como parte de la ruta.

Es decir, que la liga [http://localhost:5000/busca/Ramos](http://localhost:5000/busca/Ramos) regresará el resultado de la búsqueda de la cadena _Ramos_ en formato JSON.

**Advertencia:** Una vez ejecutado el siguente código es necesario reiniciar el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
import json
from flask import Flask, jsonify
campos = ('Nombre', 'Primer Apellido', 'Segundo Apellido')
ruta = 'data/alumnos.txt'


app =Flask(__name__)

@app.route('/')
def inicio():
    return('<p>Hola, Mundo.</p>')
    
    
@app.route('/busca/<termino>')
def busqueda(termino):
    return(jsonify(buscar_archivo(str(termino), ruta, campos)))

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run(host="0.0.0.0", port=5000)

## Ejemplo de búsqueda que regresa datos con Jinja.

Para este ejemplo se creó la plantilla [templates/busqueda.html](templates/busqueda.html), que contiene el siguiente código:

``` html
<h1> Alumnos Encontrados</h1>
<ul>
{%for alumno in alumnos %}
<li>{% for campo in ['Nombre', 'Primer Apellido', 'Segundo Apellido'] %}
{{alumno[campo]}}
{% endfor %}</li> 
{% endfor %}
</ul>
```

El siguiente código levantará un servicio web que regresará una búsqueda con la función buscar_archivo() si la cadena de búsqueda se ingresa como parte de la ruta.

Es decir, que la liga http://localhost:5000/busca/Ramos regresará el resultado de la búsqueda de la cadena _Ramos_ en formato HTML a partir de la plantilla *templates/busqueda.html*.

**Advertencia:** Una vez ejecutado el siguente código es necesario reiniciar el kernel de Jupyter para poder ejecutar el resto de las celdas de la notebook.

In [ ]:
import jinja2
from flask import Flask, render_template

campos = ('Nombre', 'Primer Apellido', 'Segundo Apellido')
ruta = 'data/alumnos.txt'

app =Flask(__name__)

@app.route('/')
def inicio():
    return('<p>Hola, Mundo.</p>')
    
    
@app.route('/busca/<termino>')
def busqueda(termino):
    return render_template('busqueda.html', alumnos=buscar_archivo(str(termino), ruta, campos))

#Si no se define el parámetro host, flask sólo será visible desde localhost
# app.run(host='localhost')
app.run(host="0.0.0.0", port=5000)

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2018.</p>